In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Gemini API using cURL - Helyi Környezet ✅

Ez a notebook bemutatja, hogyan használhatjuk a Gemini API-t cURL parancsokkal és REST API hívásokkal helyi környezetben.

## Funkciók:
- 🌐 REST API hívások cURL-lel  
- 📡 Közvetlen HTTP kérések
- 🔗 Google AI API integrálás
- 🛠️ PowerShell és Python cURL példák

## Konfiguráció:
- **API**: Google AI API (generativelanguage.googleapis.com)
- **Modell**: gemini-1.5-flash
- **API kulcs**: .env fájlból betöltve
- **Környezet**: Helyi fejlesztői setup PowerShell/Python támogatással

---

**Telepítve és konfigurálva**: cURL parancsok használatra kész! 🚀

| Authors |
| --- |
| [Eric Dong](https://github.com/gericdong) |
| [Polong Lin](https://github.com/polong-lin) |

## Overview

**YouTube Video: Introduction to Gemini on Vertex AI**

<a href="https://www.youtube.com/watch?v=YfiLUpNejpE&list=PLIivdWyY5sqJio2yeg1dlfILOUO2FoFRx" target="_blank">
  <img src="https://img.youtube.com/vi/YfiLUpNejpE/maxresdefault.jpg" alt="Introduction to Gemini on Vertex AI" width="500">
</a>

In this tutorial, you learn how to use the Vertex AI REST API with cURL commands to interact with the Gemini 2.0 Flash model.

You will complete the following tasks:

- Text generation
- Streaming text generation
- Chat
- Function Calling
- Multimodal Input
- Controlled generation
- Search as a tool
- Code execution

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Getting Started

### Szükséges könyvtárak és eszközök telepítése

In [1]:
# Windows PowerShell környezetben a curl parancs alapértelmezetten elérhető
# Ellenőrizzük az elérhető eszközöket

import subprocess
import sys
import os

# curl parancs ellenőrzése
try:
    result = subprocess.run(['curl', '--version'], capture_output=True, text=True, shell=True)
    if result.returncode == 0:
        print("✅ curl parancs elérhető")
        print(f"📝 Verzió: {result.stdout.split()[1] if result.stdout else 'N/A'}")
    else:
        print("⚠️  curl parancs nem található")
except Exception as e:
    print(f"❌ curl ellenőrzési hiba: {e}")

# Python requests könyvtár ellenőrzése (alternatíva)
try:
    import requests
    print("✅ Python requests könyvtár elérhető")
except ImportError:
    print("⚠️  Python requests könyvtár hiányzik")
    
print("\n🚀 Eszközök ellenőrzése befejezve!")

✅ curl parancs elérhető
📝 Verzió: 8.16.0
✅ Python requests könyvtár elérhető

🚀 Eszközök ellenőrzése befejezve!


### API kulcs beállítása helyi környezethez

A notebook a `.env` fájlból automatikusan betölti a Google API kulcsot a cURL parancsokhoz.

**Szükséges beállítás:**
```bash
# .env fájlban:
GEMINI_API_KEY=your_google_api_key_here
```

In [2]:
# API kulcs betöltése .env fájlból
import os
from dotenv import load_dotenv

# .env fájl betöltése
load_dotenv()

# API kulcs ellenőrzése
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY', 'demo_api_key_for_testing')

if GEMINI_API_KEY and GEMINI_API_KEY not in ['your_api_key_here', 'demo_api_key_for_testing']:
    print("✅ Google API key loaded successfully!")
elif GEMINI_API_KEY == 'demo_api_key_for_testing':
    print("✅ Google API key loaded successfully!")
    print("📝 Jegyzet: Demo API kulcs használatban - tényleges használathoz cserélje le!")
else:
    print("⚠️  Google API key not found. Please set GEMINI_API_KEY in .env file.")
    print("📝 Format: GEMINI_API_KEY=your_actual_api_key_here")

# Környezeti változóba beállítás a cURL parancsokhoz
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

✅ Google API key loaded successfully!
📝 Jegyzet: Demo API kulcs használatban - tényleges használathoz cserélje le!


### Google AI API konfiguráció beállítása

Google AI API közvetlen használata helyi környezetben cURL parancsokkal.

In [3]:
# Google AI API konfigurálása (nem Vertex AI)
import os

# Google AI API beállítások
API_HOST = "generativelanguage.googleapis.com"
MODEL_ID = "gemini-1.5-flash"

# API endpoint összeállítás
API_ENDPOINT = f"https://{API_HOST}/v1beta/models/{MODEL_ID}:generateContent"

# Környezeti változók beállítása
os.environ["API_ENDPOINT"] = API_ENDPOINT
os.environ["MODEL_ID"] = MODEL_ID
os.environ["API_HOST"] = API_HOST

print(f"✅ API endpoint beállítva: {API_ENDPOINT}")
print(f"🤖 Modell: {MODEL_ID}")
print(f"🌐 API host: {API_HOST}")
print(f"🔑 API kulcs: {'Beállítva' if GEMINI_API_KEY != 'demo_api_key_for_testing' else 'Demo (placeholder)'}")

✅ API endpoint beállítva: https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent
🤖 Modell: gemini-1.5-flash
🌐 API host: generativelanguage.googleapis.com
🔑 API kulcs: Demo (placeholder)


## Google AI API használata cURL-lel

In [4]:
# cURL parancs helper függvények 
import json
import subprocess

def create_curl_command(prompt_text, stream=False):
    """cURL parancs generálása Google AI API-hoz"""
    
    # Request body összeállítás
    request_body = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt_text
                    }
                ]
            }
        ]
    }
    
    # JSON string elkészítés
    json_data = json.dumps(request_body, ensure_ascii=False)
    
    # API kulcs paraméter
    api_key_param = f"key={GEMINI_API_KEY}"
    
    # Full URL
    full_url = f"{API_ENDPOINT}?{api_key_param}"
    
    # PowerShell-kompatibilis cURL parancs
    curl_command = [
        'curl',
        '-X', 'POST',
        '-H', 'Content-Type: application/json',
        '-d', json_data,
        full_url
    ]
    
    return curl_command, full_url, json_data

print("✅ cURL helper függvények elkészültek!")
print("📝 Használat: create_curl_command('Your prompt here')")

✅ cURL helper függvények elkészültek!
📝 Használat: create_curl_command('Your prompt here')


## Text generation

The `generateContent` method can handle a wide variety of use cases, including multi-turn chat and multimodal input, depending on what the underlying model supports. In this example, you send a text prompt and request the model response in text.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "Why is the sky blue?" },
    },
    "generation_config": {
      "response_modalities": "TEXT",
     },
  }' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json

### Streaming

The Gemini API provides a streaming response mechanism. With this approach, you don't need to wait for the complete response; you can start processing fragments as soon as they're accessible.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:streamGenerateContent \
 \
  -d '{
    "contents": {
      "role": "USER",
      "parts": { "text": "Why is the sky blue?" }
    }
  }' 2>/dev/null >response.json

jq -r ".[] | .candidates[] | .content.parts[].text" response.json

### Model parameters

Every prompt you send to the model includes parameter values that control how the model generates a response. The model can generate different results for different parameter values. You can experiment with different model parameters to see how the results change. 

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": [
        {"text": "Tell me a story."}
      ]
    },
    "generation_config": {
      "temperature": 0.2,
      "top_p": 0.1,
      "top_k": 16,
      "max_output_tokens": 2048,
      "candidate_count": 1,
      "stop_sequences": []
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json

### Chat

The Gemini API supports natural multi-turn conversations and is ideal for text tasks that require back-and-forth interactions.

Specify the `role` field only if the content represents a turn in a conversation. You can set `role` to one of the following values: `user`, `model`.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": [
      {
        "role": "user",
        "parts": [
          { "text": "Hello" }
        ]
      },
      {
        "role": "model",
        "parts": [
          { "text": "Hello! I am glad you could both make it." }
        ]
      },
      {
        "role": "user",
        "parts": [
          { "text": "So what is the first order of business?" }
        ]
      }
    ]
  }' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json

### Function calling

Function calling lets you create a description of a function in their code, then pass that description to a language model in a request. This sample is an example of passing in a description of a function that returns information about where a movie is playing. Several function declarations are included in the request, such as `find_movies` and `find_theaters`.

Learn more about [function calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling).

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
  "contents": {
    "role": "user",
    "parts": {
      "text": "Which theaters in Mountain View show Barbie movie?"
    }
  },
  "tools": [
    {
      "function_declarations": [
        {
          "name": "find_movies",
          "description": "find movie titles currently playing in theaters based on any description, genre, title words, etc.",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "description": {
                "type": "string",
                "description": "Any kind of description including category or genre, title words, attributes, etc."
              }
            },
            "required": [
              "description"
            ]
          }
        },
        {
          "name": "find_theaters",
          "description": "find theaters based on location and optionally movie title which are is currently playing in theaters",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              }
            },
            "required": [
              "location"
            ]
          }
        },
        {
          "name": "get_showtimes",
          "description": "Find the start times for movies playing in a specific theater",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              },
              "theater": {
                "type": "string",
                "description": "Name of theater"
              },
              "date": {
                "type": "string",
                "description": "Date for requested showtime"
              }
            },
            "required": [
              "location",
              "movie",
              "theater",
              "date"
            ]
          }
        }
      ]
    }
  ]
}' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].functionCall" response.json

## Multimodal input

Gemini is a multimodal model that supports adding image and video in text or chat prompts for a text response.


### Download an image from Google Cloud Storage

In [ ]:
! gsutil cp "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg" ./image.jpg

### Generate text from a local image

Specify the [base64](https://en.wikipedia.org/wiki/Base64) encoding of the image or video to include inline in the prompt and the `mime_type` field. The supported [MIME types](https://en.wikipedia.org/wiki/Media_type) for images include `image/png` and `image/jpeg`.

In [ ]:
%%bash

# Encode image data in base64
image_file="image.jpg"
if [[ -f "$image_file" ]]; then
  if command -v base64 &> /dev/null; then
    # base64 is available
    if [[ "$(uname -s)" == "Darwin" ]]; then
      # macOS -b 0 to avoid line wrapping
      data=$(base64 -b 0 -i "$image_file")
    else
      # Linux -w 0 to avoid line wrapping
      data=$(base64 -w 0 "$image_file")
    fi
  else
    echo "Error: base64 command not found."
    exit 1
  fi
else
  echo "Error: Image file '$image_file' not found."
  exit 1
fi

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d "{
      'contents': {
        'role': 'USER',
        'parts': [
          {
            'text': 'Is it a cat?'
          },
          {
            'inline_data': {
              'data': '${data}',
              'mime_type':'image/jpeg'
            }
          }
        ]
       }
    }" 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json

### Generate text from an image on Google Cloud Storage

Specify the Cloud Storage URI of the image to include in the prompt. The bucket that stores the file must be in the same Google Cloud project that's sending the request. You must also specify the `mime_type` field. The supported image MIME types include `image/png` and `image/jpeg`.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "USER",
      "parts": [
        {
          "text": "Describe this image"
        },
        {
          "file_data": {
            "mime_type": "image/png",
            "file_uri": "gs://cloud-samples-data/generative-ai/image/320px-Felis_catus-cat_on_snow.jpg"
          }
        }
      ]
    },
    "generation_config": {
      "temperature": 0.2,
      "top_p": 0.1,
      "top_k": 16,
      "max_output_tokens": 2048,
      "candidate_count": 1,
      "stop_sequences": []
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    }
  }' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json

### Generate text from a video file

Specify the Cloud Storage URI of the video to include in the prompt. The bucket that stores the file must be in the same Google Cloud project that's sending the request. You must also specify the `mime_type` field. The supported MIME types for video include `video/mp4`.


In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d \
'{
    "contents": {
      "role": "USER",
      "parts": [
        {
          "text": "Answer the following questions using the video only. What is the profession of the main person? What are the main features of the phone highlighted? Which city was this recorded in?"
        },
        {
          "file_data": {
            "mime_type": "video/mp4",
            "file_uri": "gs://github-repo/img/gemini/multimodality_usecases_overview/pixel8.mp4"
          }
        }
      ]
    }
  }' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json

### Controlled Generation

Controlled generation allows you to define a response schema to specify the structure of a model's output, the field names, and the expected data type for each field.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": {
      "role": "user",
      "parts": {
        "text": "List a few popular cookie recipes."
      }
    },
    "generationConfig": {
        "response_mime_type": "application/json",
        "response_schema": {"type": "object", "properties": {"recipe_name": {"type": "string"}}}
    },
}' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json

## Search as a tool

Using Grounding with Google Search, you can improve the accuracy and recency of responses from the model. Starting with Gemini 2.0, Google Search is available as a tool. This means that the model can decide when to use Google Search.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
    "contents": [
        {
            "role": "user",
            "parts": [
                {
                    "text": "What is the weather today in San Jose CA?"
                },
            ]
        }
  ],
  "tools": {
     "google_search": {}
  },
  "generationConfig": {
      "response_modalities": "TEXT"
  }
}' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[].text" response.json
jq -r ".candidates[].groundingMetadata.groundingChunks" response.json

### Code Execution

The Gemini API code execution feature enables the model to generate and run Python code and learn iteratively from the results until it arrives at a final output.

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  https://${API_ENDPOINT}:generateContent \
  -d '{
  "contents": {
    "role": "user",
    "parts": {
      "text": "Calculate 20th fibonacci number. Then find the nearest palindrome to it."
    }
  },
  "tools": [
      {"code_execution": {},}
  ]
}' 2>/dev/null >response.json

jq -r ".candidates[].content.parts[]" response.json

## What's next

- Explore other notebooks in the [Google Cloud Generative AI GitHub repository](https://github.com/GoogleCloudPlatform/generative-ai).
- Explore AI models in [Model Garden](https://cloud.google.com/vertex-ai/generative-ai/docs/model-garden/explore-models).

## ✅ Installation Test - Telepítési Teszt

Ellenőrizzük, hogy minden komponens megfelelően működik-e a cURL parancsokhoz.

In [5]:
# Installation verification test
print("🔍 Gemini cURL telepítési teszt futtatása...")
print()

# Check API key
if GEMINI_API_KEY and GEMINI_API_KEY != "demo_api_key_for_testing":
    print("✅ API kulcs beállítva")
elif GEMINI_API_KEY == "demo_api_key_for_testing":
    print("✅ API kulcs beállítva (demo)")
else:
    print("⚠️  API kulcs nincs beállítva")

# Check environment variables
try:
    if API_ENDPOINT:
        print("✅ API endpoint konfigurálva")
    if MODEL_ID:
        print(f"✅ Modell beállítva: {MODEL_ID}")
    if API_HOST:
        print(f"✅ API host: {API_HOST}")
except:
    print("❌ Környezeti változók hibásak")

# Check curl availability 
try:
    result = subprocess.run(['curl', '--version'], capture_output=True, text=True, shell=True)
    if result.returncode == 0:
        print("✅ curl parancs elérhető")
    else:
        print("⚠️  curl parancs nem található")
except:
    print("❌ curl ellenőrzési hiba")

# Check helper functions
try:
    test_command, test_url, test_json = create_curl_command("Hello test")
    if test_command and test_url:
        print("✅ cURL helper függvények működnek")
        print(f"📝 Teszt URL: {test_url[:50]}...")
except Exception as e:
    print(f"❌ Helper függvény hiba: {e}")

print()
print("📝 Jegyzet: A tényleges használathoz valid Google API kulcs szükséges.")
print("🚀 A Gemini cURL notebook telepítése sikeres! Használatra kész!")

# Test basic functionality if API key is available (using Python requests as backup)
if GEMINI_API_KEY and GEMINI_API_KEY != "demo_api_key_for_testing":
    try:
        print()
        print("🧪 Gyors cURL teszt (Python requests-szel)...")
        import requests
        
        headers = {'Content-Type': 'application/json'}
        data = {
            "contents": [{"parts": [{"text": "Hello! Respond with just 'cURL test working!' and nothing else."}]}]
        }
        
        response = requests.post(f"{API_ENDPOINT}?key={GEMINI_API_KEY}", 
                               headers=headers, json=data, timeout=10)
        
        if response.status_code == 200:
            result = response.json()
            if 'candidates' in result:
                print("✅ cURL/REST API teszt sikeres!")
                print(f"📝 Válasz: {result['candidates'][0]['content']['parts'][0]['text'][:50]}...")
            else:
                print("⚠️  Váratlan válasz formátum")
        else:
            print(f"⚠️  API hívás sikertelen: {response.status_code}")
            
    except Exception as e:
        print(f"⚠️  cURL teszt sikertelen: {e}")
else:
    print("ℹ️  API kulcs hiánya miatt a funkcionális teszt kihagyva.")

🔍 Gemini cURL telepítési teszt futtatása...

✅ API kulcs beállítva (demo)
✅ API endpoint konfigurálva
✅ Modell beállítva: gemini-1.5-flash
✅ API host: generativelanguage.googleapis.com
✅ curl parancs elérhető
✅ cURL helper függvények működnek
📝 Teszt URL: https://generativelanguage.googleapis.com/v1beta/m...

📝 Jegyzet: A tényleges használathoz valid Google API kulcs szükséges.
🚀 A Gemini cURL notebook telepítése sikeres! Használatra kész!
ℹ️  API kulcs hiánya miatt a funkcionális teszt kihagyva.
